In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
%cd MyDrive/Senti_Data


/content/drive/MyDrive/Senti_Data


In [16]:
pip install -U numpy==1.18.5

     |████████████████████████████████| 15.1 MB 757 kB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
df = pd.read_csv('senti_train2.csv')
df.head()

,Unnamed: 0,text,sentiment
0,0,bl theke apnader sathe sorasori kotha bolbo p...,1
1,1,Achcha amar gp sim ta off na on ei ta dekar c...,1
2,2,5 tk 500 mb dial *5000*150# offer ta koto din...,1
3,3,gp te 78 takar recharge ki hobe,1
4,4,gp te advance balance die mb kina jai na kn? ...,0


In [2]:
review_df = df[['text','sentiment']]
print(review_df.shape)
review_df.head(5)

(8497, 2)


,text,sentiment
0,bl theke apnader sathe sorasori kotha bolbo p...,1
1,Achcha amar gp sim ta off na on ei ta dekar c...,1
2,5 tk 500 mb dial *5000*150# offer ta koto din...,1
3,gp te 78 takar recharge ki hobe,1
4,gp te advance balance die mb kina jai na kn? ...,0


In [3]:
review_df["sentiment"].value_counts()

1    6493
0    2004
Name: sentiment, dtype: int64

In [4]:
sentiment_label = review_df.sentiment.factorize()
print(sentiment_label[0][:10])

[0 0 0 0 1 1 0 1 0 1]


In [5]:
tweet = review_df.text.values

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(tweet)

In [7]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=300)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 128
model = Sequential()
model.add(Embedding(10000, embedding_vector_length, input_length=300))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 128)          1280000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 300, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                35800     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 1,315,851
Trainable params: 1,315,851
Non-trainable params: 0
_________________________________________________________________
None


In [10]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=4, batch_size=32)

Train on 6797 samples, validate on 1700 samples
Epoch 1/4
6797/6797 [==============================] - 167s 24ms/sample - loss: 0.4640 - accuracy: 0.7921 - val_loss: 0.3613 - val_accuracy: 0.8324
Epoch 2/4
6797/6797 [==============================] - 167s 25ms/sample - loss: 0.2643 - accuracy: 0.8938 - val_loss: 0.2162 - val_accuracy: 0.9182
Epoch 3/4
6797/6797 [==============================] - 171s 25ms/sample - loss: 0.1597 - accuracy: 0.9425 - val_loss: 0.1176 - val_accuracy: 0.9618
Epoch 4/4
6797/6797 [==============================] - 172s 25ms/sample - loss: 0.0996 - accuracy: 0.9666 - val_loss: 0.0718 - val_accuracy: 0.9759


In [12]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=300)
    prediction = int(model.predict(tw).round().item())
    # print(prediction)
    print("Predicted label: ", sentiment_label[1][prediction])
test_sentence1 = "vai redmi k40 ase available ?"
predict_sentiment(test_sentence1)
test_sentence2 = "bhai galaxy note 20 niyen na. slow hoye jay"
predict_sentiment(test_sentence2)

Predicted label:  1
Predicted label:  0


In [19]:
testset15 = ['Poco f3 ni?',
'poco x3 Ata ki made by India ?',
'redmi Note 10 4/64',
'waiting for iphone 12 pro max',
'iphone x 5G version?',
'Poco x3 pro int. Global available? Price?',
'oneplus 8 Official naki unofficial asbe....',
'R u bringing 4g or 5g iphone 12?',
'Redmi note 9 pro max prise koto..',
'poco x3 Price??',
'When come mi 11 and price',
'pixel 4 4G or 5G ? 5G hole amar ekta lagbei lagbe',
'poco x3 ind (6/128gb) unofficial available??',
'Price 25k we niche house neya jai..',
'Price 53k er moddhe galaxy a5 pawa jabe ?']

In [20]:
for x in testset15:
    print(x)
    predict_sentiment(x)

Poco f3 ni?
Predicted label:  1
poco x3 Ata ki made by India ?
Predicted label:  0
redmi Note 10 4/64
Predicted label:  1
waiting for iphone 12 pro max
Predicted label:  1
iphone x 5G version?
Predicted label:  1
Poco x3 pro int. Global available? Price?
Predicted label:  1
oneplus 8 Official naki unofficial asbe....
Predicted label:  1
R u bringing 4g or 5g iphone 12?
Predicted label:  1
Redmi note 9 pro max prise koto..
Predicted label:  1
poco x3 Price??
Predicted label:  1
When come mi 11 and price
Predicted label:  1
pixel 4 4G or 5G ? 5G hole amar ekta lagbei lagbe
Predicted label:  1
poco x3 ind (6/128gb) unofficial available??
Predicted label:  1
Price 25k we niche house neya jai..
Predicted label:  1
Price 53k er moddhe galaxy a5 pawa jabe ?
Predicted label:  1
